In [1]:
%cd ..

/mnt/SSD_Data/active_projects/transformer_to_lstm


In [2]:
import dataclasses
from pathlib import Path
import warnings

import nlp
import torch
import numpy as np
import torch.nn.functional as F
from transformers import BertForSequenceClassification
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import train_test_split

from pytorch_helper_bot import (
    BaseBot, MovingAverageStatsTrackerCallback,  CheckpointCallback,
    LearningRateSchedulerCallback, MultiStageScheduler, Top1Accuracy,
    LinearLR, Callback
)

from nobita.models import get_sequence_model


try:
    from apex import amp
    APEX_AVAILABLE = True
except ModuleNotFoundError:
    APEX_AVAILABLE = False
# APEX_AVAILABLE = False

In [3]:
CACHE_DIR = Path("cache/")
CACHE_DIR.mkdir(exist_ok=True)

Reference:

    * https://github.com/huggingface/nlp/blob/master/notebooks/Overview.ipynb

In [4]:
class SST2Dataset(torch.utils.data.Dataset):
    def __init__(self, entries_dict):
        super().__init__()
        self.entries_dict = entries_dict
    
    def __len__(self):
        return len(self.entries_dict["label"])
    
    def __getitem__(self, idx):
        return (
            self.entries_dict["input_ids"][idx],
            self.entries_dict["attention_mask"][idx].sum(), #input_lengths
            {
                "label": self.entries_dict["label"][idx], 
                "logits": self.entries_dict["logits"][idx]
            }
        )

In [5]:
train_dict, valid_dict, test_dict = torch.load(str(CACHE_DIR / "distill-dicts.jbl"))

In [6]:
# Instantiate a PyTorch Dataloader around our dataset
train_loader = torch.utils.data.DataLoader(SST2Dataset(train_dict), batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(SST2Dataset(valid_dict), batch_size=64, drop_last=False)
test_loader = torch.utils.data.DataLoader(SST2Dataset(test_dict), batch_size=64, drop_last=False)

In [7]:
ALPHA = 0
DISTILL_OBJECTIVE = torch.nn.MSELoss()

def distill_loss(logits, targets):
    distill_part = DISTILL_OBJECTIVE(
        logits.reshape(-1), targets["logits"].reshape(-1)
    ) / 2
    classification_part = F.cross_entropy(
        logits, targets["label"]
    )
    return ALPHA * classification_part + (1-ALPHA) * distill_part

In [8]:
bert_model = BertForSequenceClassification.from_pretrained(str(CACHE_DIR / "sst2_bert_uncased")).cpu()

In [9]:
bert_model.bert.embeddings.word_embeddings.weight.shape

torch.Size([30522, 768])

In [10]:
# Note: apex does not support weight dropping
model = get_sequence_model(
    voc_size=bert_model.bert.embeddings.word_embeddings.weight.shape[0],
    emb_size=bert_model.bert.embeddings.word_embeddings.weight.shape[1],
    pad_idx = 0,
    dropoute = 0,
    rnn_hid = 768,
    rnn_layers = 3,
    bidir = True,
    dropouth = 0.25,
    dropouti = 0.25,
    wdrop = 0,
    unit_type = "lstm",
    fcn_layers = [512, 2],
    fcn_dropouts = [0.25, 0.25],
    use_attention = True
)

In [11]:
model

SequenceModel(
  (embeddings): BasicEmbeddings(
    (encoder): Embedding(30522, 768, padding_idx=0)
  )
  (encoder): RNNStack(
    (rnns): ModuleList(
      (0): LSTM(768, 384, bidirectional=True)
      (1): LSTM(768, 384, bidirectional=True)
      (2): LSTM(768, 384, bidirectional=True)
    )
    (dropouti): LockedDropout()
    (dropouths): ModuleList(
      (0): LockedDropout()
      (1): LockedDropout()
      (2): LockedDropout()
    )
  )
  (fcn): AttentionFCN(
    (attention): Attention(768, return attention=False)
    (layers): ModuleList(
      (0): LinearBlock(
        (lin): Linear(in_features=768, out_features=512, bias=True)
        (drop): Dropout(p=0.25, inplace=False)
        (bn): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): LinearBlock(
        (lin): Linear(in_features=512, out_features=2, bias=True)
        (drop): Dropout(p=0.25, inplace=False)
        (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, 

In [12]:
# Copy the embedding weights to the LSTM model
try:
    model.embeddings.encoder.emb.weight.data = bert_model.bert.embeddings.word_embeddings.weight.data
except:
    model.embeddings.encoder.weight.data = bert_model.bert.embeddings.word_embeddings.weight.data

In [13]:
# Freeze the embedding layer
for param in model.embeddings.encoder.parameters():
    param.requires_grad = False

In [14]:
model = model.cuda()

In [15]:
# Use only leaf tensors
parameters = [x for x in model.parameters() if x.is_leaf and x.requires_grad]

In [16]:
del bert_model

In [17]:
optimizer = torch.optim.Adam(parameters, lr=1e-3, betas=(0.8, 0.99))
# optimizer = torch.optim.RMSprop(parameters, lr=0.01)

In [18]:
if APEX_AVAILABLE:
    model, optimizer = amp.initialize(
        model, optimizer, opt_level="O1"
    )

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [19]:
class TransposeCallback(Callback):
    def on_batch_inputs(self, bot, input_tensors, targets):
        input_tensors = [input_tensors[0].transpose(1, 0), input_tensors[1]]
        return input_tensors, targets

In [20]:
class DistillTop1Accuracy(Top1Accuracy):
    def __call__(self, truth, pred):
        truth = truth["label"]
        return super().__call__(truth, pred)

In [21]:
total_steps = len(train_loader) * 10

checkpoints = CheckpointCallback(
    keep_n_checkpoints=1,
    checkpoint_dir=CACHE_DIR / "distill_model_cache/",
    monitor_metric="loss"
)
lr_durations = [
    int(total_steps*0.2),
    int(np.ceil(total_steps*0.8))
]
break_points = [0] + list(np.cumsum(lr_durations))[:-1]
callbacks = [
    MovingAverageStatsTrackerCallback(
        avg_window=len(train_loader) // 8,
        log_interval=len(train_loader) // 10
    ),
    LearningRateSchedulerCallback(
        MultiStageScheduler(
            [
                LinearLR(optimizer, 0.01, lr_durations[0]),
                CosineAnnealingLR(optimizer, lr_durations[1])
            ],
            start_at_epochs=break_points
        )
    ),
    checkpoints,
    TransposeCallback()
]
    
bot = BaseBot(
    log_dir = CACHE_DIR / "distill_logs",
    model=model, 
    train_loader=train_loader,
    valid_loader=valid_loader, 
    clip_grad=10.,
    optimizer=optimizer, echo=True,
    criterion=distill_loss,
    callbacks=callbacks,
    pbar=False, use_tensorboard=False,
    use_amp=APEX_AVAILABLE,
    metrics=(DistillTop1Accuracy(),)
)

[INFO][06/15/2020 21:17:52] SEED: 9293
[INFO][06/15/2020 21:17:52] # of parameters: 34,474,242
[INFO][06/15/2020 21:17:52] # of trainable parameters: 11,033,346


In [22]:
print(total_steps)

bot.train(
    total_steps=total_steps,
    checkpoint_interval=len(train_loader) // 2
)
bot.load_model(checkpoints.best_performers[0][1])
checkpoints.remove_checkpoints(keep=0)

[INFO][06/15/2020 21:17:52] Optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.8, 0.99)
    eps: 1e-08
    initial_lr: 0.001
    lr: 0.001
    weight_decay: 0
)
[INFO][06/15/2020 21:17:52] Batches per epoch: 1053
[INFO][06/15/2020 21:17:52] ====================Epoch 1====================


10530
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


/data/miniconda3/envs/rapids/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0


[INFO][06/15/2020 21:17:55] Step   105 | loss 6.16087710 | lr: 5.94e-05 | 0.029s per step
[INFO][06/15/2020 21:17:58] Step   210 | loss 5.65576329 | lr: 1.09e-04 | 0.028s per step
[INFO][06/15/2020 21:18:01] Step   315 | loss 4.55591074 | lr: 1.58e-04 | 0.028s per step
[INFO][06/15/2020 21:18:04] Step   420 | loss 3.85056060 | lr: 2.08e-04 | 0.028s per step
[INFO][06/15/2020 21:18:07] Step   525 | loss 3.42229741 | lr: 2.57e-04 | 0.028s per step
[INFO][06/15/2020 21:18:07] Metrics at step 526:
[INFO][06/15/2020 21:18:07] loss: 4.25938623
[INFO][06/15/2020 21:18:07] accuracy: 64.45%
[INFO][06/15/2020 21:18:10] Step   630 | loss 3.09515045 | lr: 3.06e-04 | 0.030s per step
[INFO][06/15/2020 21:18:13] Step   735 | loss 2.82003185 | lr: 3.56e-04 | 0.027s per step
[INFO][06/15/2020 21:18:16] Step   840 | loss 2.69006129 | lr: 4.05e-04 | 0.028s per step
[INFO][06/15/2020 21:18:19] Step   945 | loss 2.63054623 | lr: 4.54e-04 | 0.028s per step
[INFO][06/15/2020 21:18:22] Step  1050 | loss 2.477

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0


[INFO][06/15/2020 21:18:28] Step  1260 | loss 2.48394494 | lr: 6.03e-04 | 0.028s per step
[INFO][06/15/2020 21:18:31] Step  1365 | loss 2.35958329 | lr: 6.52e-04 | 0.028s per step
[INFO][06/15/2020 21:18:34] Step  1470 | loss 2.63045133 | lr: 7.01e-04 | 0.028s per step
[INFO][06/15/2020 21:18:37] Step  1575 | loss 2.25816733 | lr: 7.51e-04 | 0.029s per step
[INFO][06/15/2020 21:18:37] Metrics at step 1578:
[INFO][06/15/2020 21:18:37] loss: 1.88999063
[INFO][06/15/2020 21:18:37] accuracy: 79.59%
[INFO][06/15/2020 21:18:40] Step  1680 | loss 2.20779258 | lr: 8.00e-04 | 0.031s per step
[INFO][06/15/2020 21:18:43] Step  1785 | loss 2.05215853 | lr: 8.50e-04 | 0.027s per step
[INFO][06/15/2020 21:18:46] Step  1890 | loss 1.88951645 | lr: 8.99e-04 | 0.027s per step
[INFO][06/15/2020 21:18:49] Step  1995 | loss 1.81384740 | lr: 9.48e-04 | 0.027s per step
[INFO][06/15/2020 21:18:52] Step  2100 | loss 1.72657277 | lr: 9.98e-04 | 0.028s per step
[INFO][06/15/2020 21:18:52] Metrics at step 2104:


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0


[INFO][06/15/2020 21:19:13] Step  2835 | loss 1.52416247 | lr: 9.83e-04 | 0.029s per step
[INFO][06/15/2020 21:19:16] Step  2940 | loss 1.45761719 | lr: 9.77e-04 | 0.029s per step
[INFO][06/15/2020 21:19:19] Step  3045 | loss 1.35230513 | lr: 9.71e-04 | 0.028s per step
[INFO][06/15/2020 21:19:22] Step  3150 | loss 1.33052107 | lr: 9.64e-04 | 0.028s per step
[INFO][06/15/2020 21:19:22] Metrics at step 3156:
[INFO][06/15/2020 21:19:22] loss: 1.65857734
[INFO][06/15/2020 21:19:22] accuracy: 83.26%
[INFO][06/15/2020 21:19:22] ====================Epoch 4====================
[INFO][06/15/2020 21:19:25] Step  3255 | loss 1.25061045 | lr: 9.56e-04 | 0.029s per step
[INFO][06/15/2020 21:19:28] Step  3360 | loss 1.22404257 | lr: 9.47e-04 | 0.030s per step
[INFO][06/15/2020 21:19:31] Step  3465 | loss 1.23557551 | lr: 9.38e-04 | 0.029s per step
[INFO][06/15/2020 21:19:34] Step  3570 | loss 1.19937187 | lr: 9.28e-04 | 0.029s per step
[INFO][06/15/2020 21:19:37] Step  3675 | loss 1.19142340 | lr: 9

In [23]:
# TARGET_DIR = CACHE_DIR / "sst2_bert_uncased"
# TARGET_DIR.mkdir(exist_ok=True)
# bot.model.save_pretrained(TARGET_DIR)

In [24]:
bot.eval(valid_loader)

{'loss': (0.9946283785575026, '0.99462838'),
 'accuracy': (-0.8715596330275229, '87.16%')}

In [25]:
bot.eval(test_loader)

{'loss': (1.053325770098135, '1.05332577'),
 'accuracy': (-0.8623853211009175, '86.24%')}

In [26]:
# tokenizer.pad_token_id